<a href="https://colab.research.google.com/github/rvillat-beep/Parcial-1-2/blob/main/Parcial_2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align>

#   
# **P A R C I A L  #2**  
# **SEÑALES Y SISTEMAS**  
#

## **Ruben Dario Villa Torres**  
### **C.C. 1117132448**

</div>


# **Pregunta #1**
**Análisis teórico del demodulador AM (DSB-SC)**

Dado el sistema de demodulación:

**Señal recibida:**

$$
x(t)=A_m(t)\cos(2\pi f_c t)
$$

**Oscilador local:**

$$
\cos(2\pi f_c t)
$$

**Filtrado pasa bajas + escalamiento.**

Asumiendo

$$
\theta_0 = 0
$$

se analizan las etapas.


### **Etapa 1 — Señal recibida**

$$
x(t)=A_m(t)\cos(2\pi f_c t)
$$

**Su espectro de Fourier es:**

$$
X(f)=\tfrac{1}{2}M(f - f_c) + \tfrac{1}{2}M(f + f_c)
$$

Representa un sistema **DSB-SC** con dos bandas laterales simétricas alrededor de  
$$\pm f_c.$$


### **Etapa 2 — Mezclado con el oscilador**

$$
x(t)\cos(2\pi f_c t)
$$

Aplicamos la identidad:

$$
\cos\alpha\cos\beta=\tfrac{1}{2}\bigl[\cos(\alpha-\beta)+\cos(\alpha+\beta)\bigr]
$$

Entonces:

$$
x(t)\cos(2\pi f_c t)
= A_m(t)\tfrac{1}{2} + A_m(t)\tfrac{1}{2}\cos(4\pi f_c t)
$$

El espectro es:

$$
Y(f)=\tfrac{1}{2}M(f) + \tfrac{1}{4}M(f-2f_c) + \tfrac{1}{4}M(f+2f_c)
$$

Aparece una réplica en banda base y otra en $\pm 2f_c$.


### **Etapa 3 — Filtro paso bajas (LPF)**

Elimina las componentes en $\pm 2f_c$:

$$
y_{LPF}(t)=\tfrac{1}{2}A_m(t)
$$

Su espectro es:

$$
Y_{LPF}(f)=\tfrac{1}{2}M(f)
$$

### **Etapa 4 — Escalamiento**

Multiplicando por 2:

$$
m(t)=A_m(t)
$$

Espectro final:

$$
M(f)
$$


1. **Configuración general**

En esta parte se definen:

- La frecuencia de la portadora `fc = 20 kHz`
- El tiempo que se va a usar del audio (`30 s`)
- El nombre correcto del archivo WAV que se va a leer


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
from IPython.display import Audio

fc = 20000        # Frecuencia de portadora 20 kHz
seconds = 30      # duración que quieres usar
filename = "output.wav"  # nombre correcto del archivo

2. **Cargar archivo WAV y convertir a mono**

- Se lee el archivo WAV con `wavfile.read`
- Se convierte a `float`
- Si el audio es estéreo, se toma solo un canal (izquierdo)
- Se seleccionan **30 segundos** desde el **segundo 33** del audio original
- Se normaliza el audio entre `-1` y `1`


In [ ]:
fs, audio = wavfile.read(filename)
audio = audio.astype(float)

# Si es estéreo → pasar a mono
if audio.ndim > 1:
    audio = audio[:, 0]

# -----------------------------
# TOMAR 30 SEGUNDOS DESDE 0:33
# -----------------------------
inicio_s = 33          # segundo inicial
duracion = 30          # duración
fin_s = inicio_s + duracion

inicio = int(inicio_s * fs)
fin = int(fin_s * fs)

if fin > len(audio):
    fin = len(audio)

audio = audio[inicio:fin]

# Normalizar
audio = audio / np.max(np.abs(audio))

t = np.arange(len(audio)) / fs
m = audio  # señal mensaje

3. **Modulación AM DSB-SC**

Se genera la portadora y se multiplica por la señal mensaje:

$$
x(t) = m(t)\cos(2\pi f_c t)
$$


In [ ]:
carrier = np.cos(2*np.pi*fc*t)
x = m * carrier

4. **Demodulación: Mezclado**

Para demodular se multiplica nuevamente por la misma portadora:

$$
\text{mix}(t) = x(t)\cos(2\pi f_c t)
$$


In [ ]:
local_osc = np.cos(2*np.pi*fc*t)
mix = x * local_osc

5. **Filtro paso bajas**

Se implementa un filtro Butterworth de 6° orden con `filtfilt`:


In [ ]:
def lowpass_filter(signal, cutoff, fs):
    b, a = butter(6, cutoff/(fs/2), 'low')
    return filtfilt(b, a, signal)

demod = lowpass_filter(mix, cutoff=8000, fs=fs)

6. **Escalamiento final**

El filtro paso bajas entrega:

$$
y_{LPF}(t) = \frac{1}{2} m(t)
$$

Por eso, para recuperar la señal original, se multiplica por 2:

$$
m_{\text{rec}}(t) = 2\, y_{LPF}(t)
$$


In [ ]:
demod = 2 * demod

7. **FFT para obtener espectros**

Se usa una función auxiliar para calcular:

- Las frecuencias correspondientes a la FFT
- La magnitud del espectro normalizado


In [ ]:
def spectrum(sig, fs):
    N = len(sig)
    freq = np.linspace(-fs/2, fs/2, N)
    S = np.fft.fftshift(np.fft.fft(sig))
    return freq, np.abs(S) / np.max(np.abs(S))

f_m, M = spectrum(m, fs)
f_x, X = spectrum(x, fs)
f_mix, MIX = spectrum(mix, fs)
f_dem, DEM = spectrum(demod, fs)

#  **GRÁFICAS**


#  **Gráfica 1 — Mensaje m(t)**  
### Dominio del tiempo


In [ ]:
plt.plot(t[:2000], m[:2000], color='green')
plt.title("Mensaje m(t) en tiempo")

#  **Gráfica 2 — Espectro M(f)**


In [ ]:
plt.plot(f_m, M, color='green')
plt.title("Espectro M(f)")

#  **Gráfica 3 — Señal modulada x(t)**


In [ ]:
plt.plot(t[:2000], x[:2000], color='green')
plt.title("Señal AM x(t)")

#  **Gráfica 4 — Espectro X(f)**


In [ ]:
plt.plot(f_x, X, color='green')
plt.title("Espectro X(f)")

# **Gráfica 5 — Señal mezclada (demod step 1)**


In [ ]:
plt.plot(t[:2000], mix[:2000], color='green')
plt.title("Salida del mezclador")

# **Gráfica 6 — Espectro después del mezclador**


In [ ]:
plt.plot(f_mix, MIX, color='green')
plt.title("Espectro después del mezclador")

# **Gráfica 7 — Señal demodulada**


In [ ]:
plt.plot(t[:2000], demod[:2000], color='green')
plt.title("Señal demodulada")

#  **Gráfica 8 — Espectro demodulado**


In [ ]:
plt.plot(f_dem, DEM, color='green')
plt.title("Espectro demodulado")

# **Reproducir los 30 segundos tomados**


In [ ]:
Audio(m, rate=fs)

# **Pregunta #2**

# 1) Ecuación dinámica y función de transferencia

Tomando $y(t)$ como el desplazamiento del bloque y $F_E(t)$ como la fuerza aplicada,  
la ecuación de movimiento del sistema masa–resorte–amortiguador es:

$$
m\ddot{y}(t) + c\dot{y}(t) + ky(t) = F_E(t)
$$

Aplicando la Transformada de Laplace (condiciones iniciales cero):

$$
(ms^2 + cs + k)Y(s) = F_E(s)
$$

La función de transferencia del sistema (salida = desplazamiento, entrada = fuerza) es:

$$
G(s) = \frac{Y(s)}{F_E(s)} = \frac{1}{ms^2 + cs + k}
$$


## 2) Sistema equivalente a partir del circuito eléctrico dado

Del circuito (entrada $V_i$ en serie con $L$, nodo de salida $V_o$ con $R\parallel C$ a tierra) la impedancia en el nodo es:

$$
Z_p=\frac{1}{\frac{1}{R}+sC}=\frac{R}{1+sRC}.
$$

La ganancia $V_o/V_i$ (divisor de tensión entre $Z_p$ y $sL$) es:

$$
\frac{V_o}{V_i}=\frac{Z_p}{sL+Z_p}
=\frac{\dfrac{R}{1+sRC}}{sL+\dfrac{R}{1+sRC}}
=\frac{1}{1+\dfrac{sL}{R}(1+sRC)}.
$$

Desarrollando el denominador:

$$
\frac{V_o}{V_i}=\frac{1}{1+\dfrac{sL}{R} + s^2LC}
=\frac{1}{LC\,s^2 + \dfrac{L}{R}\,s + 1}.
$$

Normalizando el denominador se obtiene la forma canónica

$$
\frac{V_o}{V_i}=\frac{1}{LC\,s^2 + \dfrac{L}{R}\,s + 1}.
$$

Queremos que esta forma coincida con la del sistema mecánico escrita de forma adimensional (dividiendo la función de transferencia mecánica por $k$):

$$
\frac{Y(s)}{F_E(s)}=\frac{1}{ms^2+cs+k}
=\frac{1}{k}\cdot\frac{1}{\dfrac{m}{k}s^2+\dfrac{c}{k}s+1}.
$$

Comparando los coeficientes del denominador (forma con 1 en la constante) obtenemos las relaciones de equivalencia:

$$
LC=\frac{m}{k},\qquad \frac{L}{R}=\frac{c}{k}.
$$

Con estas ecuaciones puedes calcular $L,R,C$ a partir de $m,k,c$ (o al revés). Una elección práctica conveniente es fijar $L$ arbitrariamente (por ejemplo $L=1\ \text{H}$) y obtener:

$$
C=\frac{m}{kL},\qquad R=\frac{L\,k}{c}.
$$

Si $L=1$ entonces:

$$
C=\frac{m}{k},\qquad R=\frac{k}{c}.
$$

# **1. Ingreso del factor de amortiguamiento**

Se solicita al usuario ingresar manualmente el valor del factor de amortiguamiento ζ, ya que este parámetro determina el comportamiento dinámico del sistema (subamortiguado, críticamente amortiguado o sobreamortiguado).


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

# ------------------------------------------------------------
# INGRESO MANUAL DEL FACTOR DE AMORTIGUAMIENTO
# ------------------------------------------------------------

zeta = float(input("Ingrese el factor de amortiguamiento ζ: "))


# **2. Parámetros del sistema mecánico**

Se modela un sistema clásico masa–resorte utilizando los valores:

- **Masa:**  
  $$ m = 1 \, \text{kg} $$

- **Constante del resorte:**  
  $$ k = 1 \, \text{N/m} $$

A partir de estos parámetros y del factor de amortiguamiento ingresado, se calculan:

- **Frecuencia natural:**  
  $$ \omega_n = \sqrt{\frac{k}{m}} $$

- **Constante de amortiguamiento:**  
  $$ c = 2 \, \zeta \sqrt{mk} $$

- **Frecuencia amortiguada:**  
  $$ \omega_d = \omega_n \sqrt{1 - \zeta^2} \quad \text{(solo si } \zeta < 1\text{)} $$

- **Tiempos de respuesta del sistema:**
  - Tiempo pico  
  - Tiempo de levantamiento  
  - Tiempo de establecimiento aproximado  

- **Polos del sistema**, obtenidos resolviendo la ecuación característica del modelo mecánico.


In [ ]:
m = 1.0   # masa
k = 1.0   # resorte

# Calcular parámetros
omega_n = np.sqrt(k/m)
c = 2*zeta*np.sqrt(m*k)

if zeta < 1:
    omega_d = omega_n*np.sqrt(1 - zeta**2)
    t_p = np.pi / omega_d
    t_r = (np.pi - np.arccos(zeta)) / omega_d
else:
    omega_d = 0
    t_p = None
    t_r = None

t_s = 4 / (zeta * omega_n)

# Polos
A = [m, c, k]
polos = np.roots(A)

# **3. Cálculo del circuito eléctrico equivalente**

Usando la analogía mecánico–eléctrica, el sistema masa–resorte se puede representar con un circuito RLC serie.  
Los parámetros equivalentes se calculan como:

- **Inductancia:**  
  $$ L = 1 \, \text{H} $$

- **Capacitancia:**  
  $$ C = \frac{m}{kL} $$

- **Resistencia:**  
  $$ R = \frac{Lk}{c} $$

Estos valores permiten modelar el comportamiento mecánico mediante un sistema eléctrico equivalente.


In [ ]:
L = 1
C = m/(k*L)
R = L*k/c

# **5. Impresión de Resultados**

Se muestran en consola todos los valores calculados.

In [ ]:
print("\n==== RESULTADOS ====\n")
print(f"Factor de amortiguamiento ζ = {zeta}")
print(f"Constante de amortiguamiento c = {c:.4f}")
print(f"Frecuencia natural ω_n = {omega_n:.4f}")
print(f"Frecuencia amortiguada ω_d = {omega_d:.4f}")
print(f"Polos = {polos}")
print(f"Tiempo pico = {t_p}")
print(f"Tiempo de levantamiento = {t_r}")
print(f"Tiempo de establecimiento ≈ {t_s:.4f} s\n")

print("Equivalente eléctrico:")
print(f"L = 1 H")
print(f"C = {C:.4f} F")
print(f"R = {R:.4f} ohm\n")

# **5. Creación del sistema y simulación temporal**

Se construye la función de transferencia del sistema masa–resorte–amortiguador, dada por:

$$
G(s) = \frac{1}{ms^{2} + cs + k}
$$

A partir de este modelo, se simulan las principales respuestas temporales del sistema:

- **Respuesta al impulso**
- **Respuesta al escalón**
- **Respuesta a una rampa**, la cual se obtiene integrando dos veces el sistema, lo que equivale a multiplicar el denominador por \( s^2 \):
  
  $$
  G_{\text{rampa}}(s) = \frac{1}{s^{2}(ms^{2} + cs + k)}
  $$

#  **GRÁFICAS**


## **Gráfica #1 – Respuesta al Impulso**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(t_imp, y_imp, 'g')
plt.title("Respuesta al Impulso")
plt.grid()
plt.show()

## **Gráfica #2 – Respuesta al Escalon**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(t_step, y_step, 'g')
plt.title("Respuesta al Escalón")
plt.grid()
plt.show()

## **Gráfica #3 – Respuesta al Rampa**

In [ ]:
plt.figure(figsize=(12,4))
plt.plot(t_ra, y_ra, 'g')
plt.title("Respuesta a Rampa")
plt.grid()
plt.show()

## **Gráfica #4 – Polos del Sistema**

In [ ]:
plt.figure(figsize=(6,6))
plt.axhline(0, color='k')
plt.axvline(0, color='k')
plt.plot(np.real(polos), np.imag(polos), 'gx', markersize=12)
plt.title("Polos del Sistema")
plt.xlabel("Parte Real")
plt.ylabel("Parte Imaginaria")
plt.grid()
plt.show()

## **Gráfica #5 – Bode:Magnitud**

In [ ]:
w, mag, phase = signal.bode(sistema)

plt.figure(figsize=(10,4))
plt.semilogx(w, mag, 'g')
plt.title("Bode - Magnitud")
plt.grid()
plt.show()

## **Gráfica #6 – Bode:Fase**

In [ ]:
plt.figure(figsize=(10,4))
plt.semilogx(w, phase, 'g')
plt.title("Bode - Fase")
plt.grid()
plt.show()